In [ ]:
from ATARI.theory.resonance_statistics import make_res_par_avg
import ATARI.utils.hdf5 as h5io
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
import importlib
import os
from copy import copy
from ATARI.sammy_interface import sammy_classes, sammy_functions, template_creator

from ATARI.ModelData.particle_pair import Particle_Pair
from ATARI.ModelData.experimental_model import Experimental_Model

from ATARI.ModelData.measurement_models.transmission_rpi import Transmission_RPI
from ATARI.ModelData.measurement_models.capture_yield_rpi import Capture_Yield_RPI
from ATARI.syndat.syndat_model import Syndat_Model
from ATARI.syndat.control import syndatOPT
from ATARI.syndat.control import Syndat_Control
import ATARI.utils.plotting as myplot


In [ ]:
sammypath = ""
assert(sammypath != '')

In [ ]:
%matplotlib widget

## AutoFit User Example

This notebook gives an examle how the ATARI.AutoFit module can be used to automatically fit data with no need for a prior.

## First, generate data with Syndat to play with

In [ ]:
erange_all = [200, 225]
# erange_all = [795, 815]


### Reaction Model
Ta_pair = Particle_Pair(isotope = "Ta181",
                        resonance_ladder = pd.DataFrame(),
                        formalism = "XCT",
                        energy_range = erange_all,
                        ac = 0.8127,
                        M = 180.94803,
                        I = 3.5,
                        l_max = 1
)
Ta_pair.add_spin_group(Jpi='3.0',
                       J_ID=1,
                       D=9.0030,
                       gn2_avg=452.56615,
                       gn2_dof=1,
                       gg2_avg=32.0,
                       gg2_dof=1000)

Ta_pair.add_spin_group(Jpi='4.0',
                       J_ID=2,
                       D=8.3031,
                       gn2_avg=332.24347, 
                       gn2_dof=1,
                       gg2_avg=32.0,
                       gg2_dof=1000)

### Experimental Models
cap1mm_exp_gen = Experimental_Model(title = "cap1",
                                reaction ="capture", 
                                energy_range = erange_all,
                                n = (0.005631, 0),
                                FP = (45.27, 0.05),
                                burst= (8.0,1.0), 
                                temp= (294.2610, 0.0),
                                channel_widths={
                                    "maxE": [270], 
                                    "chw": [300],#[102.7],
                                    "dchw": [0.8]
                                }
                               )

trans1mm_exp_gen = Experimental_Model(title = "trans1",
                               reaction = "transmission", 
                               energy_range = erange_all,
                                n = (0.00566,0.0),  
                                FP = (100.14,0.0), 
                                burst = (8, 0.0), 
                                temp = (294.2610, 0.0),
                               channel_widths={
                                    "maxE": [270],#[216.16, 613.02], 
                                    "chw": [100],#[204.7, 102.4],
                                    "dchw": [1.6, 1.6]
                                }
                               )

sammy_rto = sammy_classes.SammyRunTimeOptions(sammypath,
                                        {"Print":   True,
                                         "bayes":   False,
                                         "keep_runDIR": False,
                                         "sammy_runDIR": "sammy_runDIR"
                                         })

template_creator.make_input_template('template_T.inp', Ta_pair, trans1mm_exp_gen, sammy_rto)
template_creator.make_input_template('template_Y.inp', Ta_pair, cap1mm_exp_gen, sammy_rto)

cap1mm_exp_gen.template = os.path.realpath('template_Y.inp')
trans1mm_exp_gen.template = os.path.realpath('template_T.inp')

### Measurement Models
cap1mm_meas_gen = Capture_Yield_RPI(trig_g     =  (1200000,   0),
                                    trig_bg    =  (100000,  0),
                                    trig_f     =  (120000,   0),
                                    trig_bf    =  (100000,  0))
cap1mm_meas_red = Capture_Yield_RPI(trig_g     =  (1200000,   0),
                                    trig_bg    =  (100000,  0),
                                    trig_f     =  (120000,   0),
                                    trig_bf    =  (100000,  0))

trans1mm_meas_gen = Transmission_RPI()
trans1mm_meas_red = Transmission_RPI()


### Syndat Models
options_cap1mm = syndatOPT(calculate_covariance=False, save_raw_data=True)

syndat_cap1mm = Syndat_Model(cap1mm_exp_gen,
                            cap1mm_meas_gen,
                            cap1mm_meas_red,
                            options = options_cap1mm,
                            title='cap1mm')

options_trans1mm = syndatOPT(calculate_covariance = True)
syndat_trans1mm = Syndat_Model(trans1mm_exp_gen,
                            trans1mm_meas_gen,
                            trans1mm_meas_red,
                            options = options_trans1mm,
                            title='trans1mm')


syndat = Syndat_Control(particle_pair= Ta_pair,
                        syndat_models= [syndat_trans1mm, syndat_cap1mm],
                        model_correlations=None,
                        options=syndatOPT()
                        )

syndat.sample(sammy_rto, num_samples=1)

In [ ]:
### unpack sample and plot

sample1 = syndat.get_sample(0)
true_ladder = copy(sample1['trans1mm'].par_true)
true_ladder['varyE'] = np.ones(len(true_ladder))
true_ladder['varyGg'] = np.ones(len(true_ladder))
true_ladder['varyGn'] = np.ones(len(true_ladder))

datasets = [sample1['cap1mm'].pw_reduced, sample1['trans1mm'].pw_reduced]#, transdat3, transdat6]
experiments= [cap1mm_exp_gen, trans1mm_exp_gen]#, exptrans3, exptrans6]
covariance_data = [sample1['cap1mm'].covariance_data, sample1['trans1mm'].covariance_data]



# fit from true
sammy_rto_fit = sammy_classes.SammyRunTimeOptions(sammypath,
                                        {"Print":   True,
                                         "bayes":   True,
                                         "keep_runDIR": False,
                                         "sammy_runDIR": "sammy_runDIR_autofit"
                                         })

sammyINPyw = sammy_classes.SammyInputDataYW(
    particle_pair = Ta_pair,
    resonance_ladder = true_ladder,
    datasets = datasets, experimental_covariance=covariance_data, experiments = experiments,
    max_steps = 5, iterations = 3, step_threshold = 0.01
    )
true_out = sammy_functions.run_sammy_YW(sammyINPyw, sammy_rto_fit)


fig = myplot.plot_datafits(datasets, experiments, 
    true_pars = true_ladder, 
    true=true_out.pw,
    fig_size=(6,4),
)

fig.tight_layout()



In [ ]:
from ATARI.AutoFit.initial_FB_solve import InitialFB, InitialFBOPT


options = InitialFBOPT(Gn_threshold=1e-1,
                       iterations=3,
                       max_steps = 20,
                       LevMarV0=0.1,
                       fit_all_spin_groups=False,
                       spin_group_keys = ['3.0'],
                       step_threshold=0.001,
                       starting_Gn1_multiplier = 50,
                       external_resonances=True
                    #    num_Elam=100
                       )
autofit_initial = InitialFB(options)

In [ ]:
outs = autofit_initial.fit(Ta_pair,
                               [200,225],
                               datasets,
                               experiments,
                               covariance_data,
                               sammy_rto_fit)

In [ ]:
final_fb_output  = outs.sammy_outs_fit_1[0]

fig = myplot.plot_datafits(datasets, experiments, 
    fits=final_fb_output.pw_post, 
    fits_chi2=final_fb_output.chi2n_post,
    fit_pars = final_fb_output.par_post,
    # title: str = '',
    show_spingroups = True,
    fig_size = (8,6)
)

In [ ]:
import importlib
from ATARI.AutoFit import chi2_eliminator_v2

importlib.reload(chi2_eliminator_v2)

start_ladder =  outs.final_resonace_ladder #true_ladder#
assert isinstance(start_ladder, pd.DataFrame)

# side resonances - keep empty
side_resonances_df = pd.DataFrame()

# defining rto & inputs

elim_sammyINPyw = sammy_classes.SammyInputDataYW(
    particle_pair = Ta_pair,
    resonance_ladder = start_ladder,

    datasets = datasets,
    experimental_covariance=covariance_data,
    experiments = experiments,

    max_steps = 5,
    iterations = 3,
    step_threshold = 0.01,
    autoelim_threshold = None,

    LS = False,
    LevMar = True,
    LevMarV = 1.5,

    minF = 1e-5,
    maxF = 10,
    initial_parameter_uncertainty = 0.05
    )

# defining the elim_opts
elim_opts = chi2_eliminator_v2.elim_OPTs(chi2_allowed = 50,
                                      fixed_resonances_df = side_resonances_df,
                                      deep_fit_max_iter = 5,
                                      deep_fit_step_thr = 0.1,
                                      start_fudge_for_deep_stage = 0.1
                                      )

In [ ]:
elimi = chi2_eliminator_v2.eliminator_by_chi2(rto=sammy_rto_fit,
                                            sammyINPyw = elim_sammyINPyw , 
                                            options = elim_opts
                            )

In [ ]:
hist = elimi.eliminate(ladder_df=start_ladder)

In [ ]:
print(f'Eliminated from {hist.ladder_IN.shape[0]} res -> {hist.ladder_OUT.shape[0]}')
minkey = min(hist.elimination_history.keys())

sammyOUT_elim = hist.elimination_history[minkey]['selected_ladder_chars']

# fig = plot(datasets, experiments, fits=sammyOUT_elim.pw_post, priors=[], true=datasets)
# fig.tight_layout()

fig = myplot.plot_datafits(datasets, experiments, 
    fits=sammyOUT_elim.pw_post, 
    fits_chi2=sammyOUT_elim.chi2n_post,
    priors=[], priors_chi2=[],
    fit_pars = sammyOUT_elim.par_post,
    # title: str = '',
    show_spingroups = True,
    fig_size = (8,6)
)

fig.tight_layout()

In [ ]:
from ATARI.AutoFit.spin_group_selection import SpinSelectOPT, SpinSelect


spinselectOPT = SpinSelectOPT(step_threshold=0.01,
                              max_steps=10)
spinselector = SpinSelect(spinselectOPT)

sammyOUT_spin = spinselector.try_all_spin_groups([1.0, 2.0],
                                                sammyOUT_elim.par_post,
                                                Ta_pair,
                                                datasets,
                                                experiments,
                                                covariance_data,
                                                sammy_rto_fit,
                                                external_resonance_indices=[])

In [ ]:
all_outs, leading_model = sammyOUT_spin

print(sammyOUT_elim.chi2n_post, np.sum(sammyOUT_elim.chi2n_post))
print(leading_model.chi2n_post, np.sum(leading_model.chi2n_post))
print(sammyOUT_elim.par_post)
print(leading_model.par_post)

In [ ]:


fig = myplot.plot_datafits(datasets, experiments, 
    fits=leading_model.pw_post, 
    fits_chi2=leading_model.chi2n_post,
    priors=[], priors_chi2=[],
    true=true_out.pw, true_chi2=[],
    true_pars = true_ladder, 
    # prior_pars = sammyOUT_elim.par_post,
    fit_pars = leading_model.par_post,
    # title: str = '',
    show_spingroups = True,
    fig_size = (8,6)
)

fig.tight_layout()